In [20]:
%run ../talktools.py

# Monitoring and Optimization of NNs

(UCB Datalab AY 128/256; 2021)

We have thus far built network architectures and trained them using guesses for the architecture and using default learning parameters. Watching the outputs and introspection of the training curves is somewhat cumbersome. What if we want to do a lot of experiments and assemble the results in one place?

Recall in previous notebooks we used a callback to store results in logfiles that can be visualized by `Tensorboard` (https://www.tensorflow.org/tensorboard):

```python
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
...
model.fit(...
          callbacks=[tensorboard_callback,...])
```

Let's look at the results in tensorboard. First install a Jupyter extension (`jupyter-tensorboard`):

```bash
$ pip install git+https://github.com/lspvic/jupyter_tensorboard.git@refs/pull/63/head
```

(You can use Tensorboard outside of Jupyter but downloading the log directories and running tensorboard directly; it's difficult to get it working in datahub.berkeley.edu unfortunately)

We already have logfiles that tensorboard can see:

<img src="data/tb.png">

Let's watch the progression in realtime:

In [6]:
import datetime
import warnings
import tensorflow as tf

warnings.simplefilter(action='ignore', category=FutureWarning)

fashion_mnist = tf.keras.datasets.fashion_mnist

(x_train, y_train),(x_test, y_test) = fashion_mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

def create_model():
    return tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dropout(0.2),
        tf.keras.layers.Dense(10, activation='softmax')
    ])

In [4]:
model = create_model()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

model.fit(x=x_train, 
          y=y_train, 
          epochs=10, 
          validation_data=(x_test, y_test), 
          callbacks=[tensorboard_callback])

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 19s 313us/sample - loss: 0.4931 - acc: 0.8233 - val_loss: 0.4242 - val_acc: 0.8495
Epoch 2/10
60000/60000 [==============================] - 20s 336us/sample - loss: 0.3815 - acc: 0.8589 - val_loss: 0.4234 - val_acc: 0.8487
Epoch 3/10
60000/60000 [==============================] - 22s 367us/sample - loss: 0.3484 - acc: 0.8711 - val_loss: 0.4205 - val_acc: 0.8487
Epoch 4/10
60000/60000 [==============================] - 19s 319us/sample - loss: 0.3292 - acc: 0.8790 - val_loss: 0.3604 - val_acc: 0.8719
Epoch 5/10
60000/60000 [==============================] - 21s 349us/sample - loss: 0.3125 - acc: 0.8836 - val_loss: 0.3563 - val_acc: 0.8684
Epoch 6/10
60000/60000 [==============================] - 17s 288us/sample - loss: 0.3003 - acc: 0.8879 - val_loss: 0.3438 - va

We can also use external services to capture training data, resource allocation, and more.

<img src="https://camo.githubusercontent.com/dd842f7b0be57140e68b2ab9cb007992acd131c48284eaf6b1aca758bfea358b/68747470733a2f2f692e696d6775722e636f6d2f52557469567a482e706e67">

"Use W&B to build better models faster. Track and visualize all the pieces of your machine learning pipeline, from datasets to production models."

In [8]:
#!pip install wandb

In [14]:
import wandb
wandb.login()

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ········


wandb: Appending key for api.wandb.ai to your netrc file: /Users/jbloom/.netrc


True

In [18]:
from wandb.keras import WandbCallback

# Step1: Initialize W&B run
run = wandb.init(project='ucb-datalab-2021', entity='profjsb')

# 2. Save model inputs and hyperparameters
config = wandb.config
config.learning_rate = 0.01

epoch,9
loss,0.49933
acc,0.82202
val_loss,0.45825
val_acc,0.8473
_runtime,123
_timestamp,1619469215
_step,9
best_val_loss,0.45825
best_epoch,9


epoch,▁▂▃▃▄▅▆▆▇█
loss,█▄▃▃▂▂▁▂▁▁
acc,▁▅▆▆▇▇▇▇██
val_loss,▆▅▅▄█▅▃▅▃▁
val_acc,▂▄▄▄▁▄▅▃▆█
_runtime,▁▂▃▄▄▅▆▇▇█
_timestamp,▁▂▃▄▄▅▆▇▇█
_step,▁▂▃▃▄▅▆▆▇█


In [17]:
model = create_model()

opt = tf.keras.optimizers.Adam(learning_rate=config.learning_rate)
model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# 3. Log layer dimensions and metrics over time
model.fit(x=x_train, 
          y=y_train, 
          epochs=10, 
          validation_data=(x_test, y_test),
          callbacks=[WandbCallback()])

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
60000/60000 [==============================] - 12s 208us/sample - loss: 0.6522 - acc: 0.7758 - val_loss: 0.5357 - val_acc: 0.8135
Epoch 2/10
60000/60000 [==============================] - 7s 122us/sample - loss: 0.5583 - acc: 0.8026 - val_loss: 0.5211 - val_acc: 0.8248
Epoch 3/10
60000/60000 [==============================] - 11s 182us/sample - loss: 0.5463 - acc: 0.8067 - val_loss: 0.5136 - val_acc: 0.8261
Epoch 4/10
60000/60000 [==============================] - 7s 114us/sample - loss: 0.5355 - acc: 0.8099 - val_loss: 0.5029 - val_acc: 0.8235
Epoch 5/10
60000/60000 [==============================] - 6s 97us/sample - loss: 0.5267 - acc: 0.8143 - val_loss: 0.5599 - val_acc: 0.8055
Epoch 6/10
60000/60000 [==============================] - 9s 152us/sample - loss: 0.5170 - acc: 0.8166 - val_loss: 0.5122 - val_acc: 0.8262
Epoch 7/10
60000/60000 [==============================] - 6s 104us/sample - loss: 0.5084 - acc: 0.8192 - val_

In [19]:
run.finish()